In [ ]:
import pandas as pd
import requests
import time

url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
df = pd.read_html(url)[0][['Symbol', 'Security']]
df.columns = ['Ticker', 'Company']

def get_wikidata_aliases(company_name, ticker):
    try:
        search_queries = [f"{company_name} {ticker}", company_name, ticker]
        for query in search_queries:
            search_url = f'https://www.wikidata.org/w/api.php?action=wbsearchentities&format=json&language=en&search={query}'
            response = requests.get(search_url).json()
            if response.get('search'):
                for result in response['search']:
                    entity_id = result['id']
                    data_url = f'https://www.wikidata.org/wiki/Special:EntityData/{entity_id}.json'
                    data = requests.get(data_url).json()
                    entity = data['entities'].get(entity_id)
                    if not entity:
                        continue

                    claims = entity.get('claims', {})
                    instance_of = claims.get('P31', [])
                    instance_ids = [claim['mainsnak']['datavalue']['value']['id']
                                    for claim in instance_of if 'datavalue' in claim['mainsnak']]
                    if 'Q4830453' in instance_ids or 'Q43229' in instance_ids:  # business / org
                        aliases = entity.get('aliases', {}).get('en', [])
                        return [a['value'] for a in aliases] if aliases else []
        return []
    except Exception as e:
        print(f"Error fetching aliases for {company_name}: {e}")
        return []

alias_list = []
for i, row in df.iterrows():
    print(f"[{i+1}/{len(df)}] Fetching aliases for: {row['Company']} ({row['Ticker']})")
    aliases = get_wikidata_aliases(row['Company'], row['Ticker'])
    print(aliases)
    alias_list.append(aliases)
    time.sleep(0.5)

df['Aliases'] = alias_list

df.to_csv('sp500_with_wikidata_aliases.csv', index=False)
print(df.head(10))


[1/503] Fetching aliases for: 3M (MMM)
['Minnesota Mining and Manufacturing', 'Minnesota Mining & Mfg. Co.', 'Minnesota Mining and Manufacturing Company', '3M corporation', '3M Company']
[2/503] Fetching aliases for: A. O. Smith (AOS)
['A. O. Smith Corporation', 'A.O. Smith', 'A.O. Smith Corporation']
[3/503] Fetching aliases for: Abbott Laboratories (ABT)
['Abbott Labs', 'Abbott Laboratories, Inc.', 'Ross Laboratories', 'Abbott Pharmaceuticals', 'Abbott Nutrition', 'I-Stat', 'I-STAT', 'Abbott Molecular', 'Isomil', 'Abbot Laboratories', 'Abbott']
[4/503] Fetching aliases for: AbbVie (ABBV)
['AbbVie Inc.', 'ABBV']
[5/503] Fetching aliases for: Accenture (ACN)
['Accenture Global Solutions Limited (Dublin)', 'Accenture (Dublin)', 'Accenture PLC']
[6/503] Fetching aliases for: Adobe Inc. (ADBE)
['Adobe Systems', 'Adobe Systems, Inc.', 'Adobe Systems Software(Beijing)Co.,Ltd.', 'ADBE', 'ADOBE INC. (San Jose, CA)', 'adobe.com', 'Adobe Inc.', 'Adobe Systems Incorporated']
[7/503] Fetching ali

In [ ]:
import pandas as pd
import re
import ast

df = pd.read_csv('sp500_with_wikidata_aliases.csv')

remove_words = [
    'The', 'Inc', 'Corporation', 'Corp', 'Ltd', 'Limited', 'Group', 'Co', 'PLC', 'LLC', 'and', 'GmbH'
]

def clean_company_name(company_name):
    company_name = re.sub(r'[.,]', '', company_name)  
    pattern = r'\b(?:' + '|'.join(re.escape(word) for word in remove_words) + r')\b'
    company_name = re.sub(pattern, '', company_name, flags=re.IGNORECASE)
    company_name = re.sub(r'\s+', ' ', company_name).strip()
    return company_name

def append_cleaned_aliases(row):
    current_aliases = row['Aliases']
    if isinstance(current_aliases, str): 
        try:
            current_aliases = ast.literal_eval(current_aliases)
        except:
            current_aliases = []
    
    cleaned_name = clean_company_name(row['Company'])
    
    if cleaned_name and not any(cleaned_name.lower() == alias.lower() for alias in current_aliases):
        current_aliases.append(cleaned_name)
        
    return current_aliases

df['Aliases'] = df.apply(append_cleaned_aliases, axis=1)

df.to_csv('sp500_with_appended_aliases.csv', index=False)

print(df[df['Company'].str.contains('Apple', case=False)][['Company', 'Aliases']])


       Company                                            Aliases
39  Apple Inc.  [Apple Computer Inc, Apple Incorporated, Apple...
